## Module 1 Homework

## Import packages and set constants

In [1]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
end = date.today()
print(f'Year = {end.year}; month= {end.month}; day={end.day}')

start = date(year=end.year-70, month=end.month, day=end.day)
print(f'Period for indexes: {start} to {end} ')

Year = 2024; month= 5; day=8
Period for indexes: 1954-05-08 to 2024-05-08 


### Question 1: [Macro] Average growth of GDP in 2023

**What is the average growth (in %) of GDP in 2023?**

Download the timeseries Real Gross Domestic Product (GDPC1) from FRED (https://fred.stlouisfed.org/series/GDPC1). 
Calculate year-over-year (YoY) growth rate (that is, divide current value to one 4 quarters ago). Find the average YoY growth in 2023 (average from 4 YoY numbers).
Round to 1 digit after the decimal point: e.g. if you get 5.66% growth => you should answer  5.7

**Notes**

Definitions:
- GDP = Gross Domestic Product
  - In Dutch: BNP = Bruto Nationaal Product is het totale inkomen van de mensen die in een land wonen, ongeacht of ze in dat land of het buitenland werken.
- BBP = Bruto Binnenlands Product, de waarde van alle goederen en diensten die in een land worden geproduceerd. 

In [3]:
# Load data
gdpc1 = pd.read_csv("data/GDPC1.csv").astype({"DATE": "datetime64[ns]"})

# Lowercase column names
gdpc1.columns = [col.lower() for col in gdpc1.columns]

# Calculate year-over-year growth: (new-old/old)*100
gdpc1["yoy"] = (gdpc1["gdpc1"]/gdpc1["gdpc1"].shift(4)-1)*100

# Print first rows
gdpc1.head()

,date,gdpc1,yoy
0,1947-01-01,2182.681,NaN
1,1947-04-01,2176.892,NaN
2,1947-07-01,2172.432,NaN
3,1947-10-01,2206.452,NaN
4,1948-01-01,2239.682,2.611513


In [4]:
# Answer question
print(
    "Q1) Average growth (in %) of GDP in 2023 is:",
    round(gdpc1[gdpc1["date"].dt.year==2023]["yoy"].sum()/4, 1)
)

Q1) Average growth (in %) of GDP in 2023 is: 2.5


**Extra: trying to find the average growth of GDP in 2023 in the Netherlands**

Definitions:
- SCA = seasonally and calendar-adjusted
- NSA = non-seasonally-adjusted.

In [5]:
# Import eurostat package
import eurostat

# Load Eurostat data
gdpeu = eurostat.get_data_df("tipsau20")

# Lowercase column names
gdpeu.columns = [col.lower() for col in gdpeu.columns]

# Rename column names
gdpeu.rename(columns={"geo\\time_period": "country"}, inplace=True)

# Drop unused columns
gdpeu.drop(columns=["unit", "na_item"], inplace=True)

# Melt data frame to transform columns to rows
gdpeu = gdpeu.melt(
    id_vars=["country", "freq", "s_adj"], var_name="date", value_name="gdp")

# Extract year and quarter from date
gdpeu["year"] = gdpeu["date"].str[:4]
gdpeu["quarter"] = gdpeu["date"].str[-1]

# Show first rows
gdpeu.head()

,country,freq,s_adj,date,gdp,year,quarter
0,AT,Q,NSA,1995-q1,41700.2,1995,1
1,BE,Q,NSA,1995-q1,51243.0,1995,1
2,BG,Q,NSA,1995-q1,268.0,1995,1
3,CY,Q,NSA,1995-q1,1743.0,1995,1
4,CZ,Q,NSA,1995-q1,351523.0,1995,1


In [6]:
def print_average_gdp_growth(
    df: pd.DataFrame, country: str = "NL", year: str = "2023"
) -> None:

    # Filter data based on input parameters
    gdp = gdpeu[
        (gdpeu["country"]==country)
        & (gdpeu["s_adj"]=="NSA")
    ]

    # Calculate year-over-year growth
    gdp["yoy"] = (gdp["gdp"]/gdp["gdp"].shift(4)-1)*100

    # Show last rows
    gdp.tail()

    print(
        f"Q1) Average growth (in %) of GDP in {year} in {country} is:",
        round(gdp[gdp["year"]==year]["yoy"].sum()/4, 1)
    )

In [7]:
# Average growth in the Netherlands
print_average_gdp_growth(gdpeu, country="NL")
# Average growth in the Italy
print_average_gdp_growth(gdpeu, country="IT")

Q1) Average growth (in %) of GDP in 2023 in NL is: 7.9
Q1) Average growth (in %) of GDP in 2023 in IT is: 6.3


### Question 2. [Macro] Inverse "Treasury Yield"

**Find the min value of (dgs10-dgs2) after since year 2000 (2000-01-01) and write it down as an answer, round to 1 digit after the decimal point.**


Download DGS2 and DGS10 interest rates series (https://fred.stlouisfed.org/series/DGS2,
 https://fred.stlouisfed.org/series/DGS10). Join them together to one dataframe on date (you might need to read about pandas.DataFrame.join()), calculate the difference dgs10-dgs2 daily.

(Additional: think about what does the "inverted yield curve" mean for the market and investors? do you see the same thing in your country/market of interest? Do you think it can be a good predictive feature for the models?)

**Notes**

"Treasury Yields" = rendementen op staatsobligaties

In [8]:
# Load data
dgs2 = pd.read_csv("data/dgs2.csv")
dgs10 = pd.read_csv("data/dgs10.csv")

# Replace . by NaN values
dgs2.replace(".", np.NaN, inplace=True)
dgs10.replace(".", np.NaN, inplace=True)

# Lowercase column names
dgs2.columns = [col.lower() for col in dgs2.columns]
dgs10.columns = [col.lower() for col in dgs10.columns]

# Set column types
dgs2 = dgs2.astype({"date": "datetime64[ns]", "dgs2": "float"})
dgs10 = dgs10.astype({"date": "datetime64[ns]", "dgs10": "float"})

# Merge DGS2 and DGS10 dataframes
dgs = dgs2.merge(dgs10, on="date", how="inner")

# Calculate daily differences between DGS2 and DGS10
dgs["diff"] = dgs["dgs10"] - dgs["dgs2"]

# Show first rows
dgs.head()

,date,dgs2,dgs10,diff
0,2000-01-03,6.38,6.58,0.20
1,2000-01-04,6.30,6.49,0.19
2,2000-01-05,6.38,6.62,0.24
3,2000-01-06,6.35,6.57,0.22
4,2000-01-07,6.31,6.52,0.21


In [9]:
print(
    "Q2) Minimal difference between DGS10 and DGS2 since year 2000:",
    round(dgs[dgs["date"]>"2000-01-01"]["diff"].min(), 1)
)

Q2) Minimal difference between DGS10 and DGS2 since year 2000: -1.1


### Question 3. [Index] Which Index is better recently?

**Compare S&P 500 and IPC Mexico indexes by the 5 year growth and write down the largest value as an answer (%)**

Download on Yahoo Finance two daily index prices for S&P 500 (^GSPC, https://finance.yahoo.com/quote/%5EGSPC/) and IPC Mexico (^MXX, https://finance.yahoo.com/quote/%5EMXX/). Compare 5Y growth for both (between 2019-04-09 and 2024-04-09). Select the higher growing index and write down the growth in % (closest integer %). E.g. if ratio end/start was 2.0925 (or growth of 109.25%), you need to write down 109 as your answer.

(Additional: think of other indexes and try to download stats and compare the growth? Do create 10Y and 20Y growth stats. What is an average yearly growth rate (CAGR) for each of the indexes you select?)

**Notes**

In [10]:
# Import yfinance package
import yfinance as yf

# Load S&P500 and IPC Mexico daily index prices data

# https://finance.yahoo.com/quote/%5EGSPC/
# SNP - SNP Real Time Price. Currency in USD
snp500 = yf.download(
    tickers="^GSPC",
    period="max",
    interval="1d").reset_index()

# https://finance.yahoo.com/quote/%5EMXX
# MXX - IPC MEXICO. Currency in USD
ipc_mexico = yf.download(
    tickers = "^MXX",
    period = "max",
    interval = "1d").reset_index()

# Calculate 5 years growth
snp500["5y_growth"] = snp500["Adj Close"] / snp500["Adj Close"].shift(365*5)
ipc_mexico["5y_growth"] = ipc_mexico["Adj Close"] / ipc_mexico["Adj Close"].shift(365*5)

# Merge 5Y growth rates together
growths = snp500[["Date", "5y_growth"]].merge(
    ipc_mexico[["Date", "5y_growth"]], 
    on="Date", 
    suffixes=("_snp", "_ipc"))

# Show first rows
growths.head()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Date,5y_growth_snp,5y_growth_ipc
0,1991-11-08,2.341000,NaN
1,1991-11-11,2.353167,NaN
2,1991-11-12,2.373983,NaN
3,1991-11-13,2.372455,NaN
4,1991-11-14,2.386145,NaN


In [11]:
# Get statistics about growth rates
growths[
    (growths["Date"] >= "2019-01-09") 
    & (growths["Date"] <= "2024-04-09")
][["5y_growth_snp", "5y_growth_ipc"]].describe()

,5y_growth_snp,5y_growth_ipc
count,1284.000000,1284.000000
mean,2.133274,1.120767
std,0.160927,0.124511
min,1.558284,0.758234
25%,2.024059,1.060081
50%,2.160428,1.146979
75%,2.244561,1.203179
max,2.517096,1.385267


In [12]:
print("Q3) Based on the 5 year growth rates, the S&P500 index seems to be outperforming the IPC Mexico index in the last years")

Q3) Based on the 5 year growth rates, the S&P500 index seems to be outperforming the IPC Mexico index in the last years


### Question 4. [Stocks OHLCV] 52-weeks range ratio (2023) for the selected stocks

**Find the largest range ratio [=(max-min)/max] of Adj.Close prices in 2023**

Download the 2023 daily OHLCV data on Yahoo Finance for top6 stocks on earnings (https://companiesmarketcap.com/most-profitable-companies/): 2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM.

Here is the example data you should see in Pandas for "2222.SR": https://finance.yahoo.com/quote/2222.SR/history

Calculate maximum-minimim "Adj.Close" price for each stock and divide it by the maximum "Adj.Close" value.
Round the result to two decimal places (e.g. 0.1575 will be 0.16)

(Additional: why this may be important for your research?)

**Notes**

Definitions:
- OHLCV = Open, High, Low, Close and Volum

In [13]:
# Define function to get Yahoo Finance code to avoid duplicate code
def get_yf_data(name: str, period: str = "max", interval: str = "1d") -> pd.DataFrame:

    # Load data
    df = yf.download(tickers=name, period=period, interval=interval).reset_index()

    # Lowercase column names
    df.columns = [col.lower() for col in df.columns]
    
    # Get year
    df["year"] = df["date"].dt.year

    return df

In [14]:
# Load data
saudi_arabian_oil_company = get_yf_data("2222.SR")
berkshire_gathaway = get_yf_data("BRK-B")
apple = get_yf_data("AAPL")
microsoft = get_yf_data("MSFT")
alphabet = get_yf_data("GOOG")
jp_morgan = get_yf_data("JPM")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [15]:
def get_largest_range_ratio(df: pd.DataFrame, year: int = 2023):

    df = df[df["year"]==year]

    return round((df["adj close"].max() - df["adj close"].min()) / df["adj close"].max(), 2)

In [16]:
print("Saudi Arabian Oil Company:", get_largest_range_ratio(saudi_arabian_oil_company))
print("Berkshire Gathaway:", get_largest_range_ratio(berkshire_gathaway))
print("Apple:", get_largest_range_ratio(apple))
print("Microsoft:", get_largest_range_ratio(microsoft))
print("Alphabet:", get_largest_range_ratio(alphabet))
print("JP Morgan:", get_largest_range_ratio(jp_morgan))

Saudi Arabian Oil Company: 0.21
Berkshire Gathaway: 0.21
Apple: 0.37
Microsoft: 0.42
Alphabet: 0.39
JP Morgan: 0.28


The maximum-minimum "Adj.Close" price metric gives an understanding of how much growth the stock has shown in the past year. The higher this value the better.

### Question 5. [Stocks] Dividend Yield
**Find the largest dividend yield for the same set of stocks**

Use the same list of companies (2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM) and download all dividends paid in 2023.
You can use `get_actions()` method or `.dividends` field in yfinance library (https://github.com/ranaroussi/yfinance?tab=readme-ov-file#quick-start)

Sum up all dividends paid in 2023 per company and divide each value by the closing price (Adj.Close) at the last trading day of the year.

Find the maximm value in % and round to 1 digit after the decimal point. (E.g., if you obtained $1.25 dividends paid and the end year stock price is $100, the dividend yield is 1.25% -- and your answer should be equal to 1.3)

In [19]:
data = pd.DataFrame()
stocks = ["2222.SR", "BRK-B", "AAPL", "MSFT", "GOOG", "JPM"]
for name in stocks:
    # Load Ticker
    yf_ticker = yf.Ticker(name)

    # Get dividends
    dividends = yf_ticker.dividends.reset_index()
    dividends["name"] = name

    # Select only dividends paid in 2023
    dividends = dividends[(dividends["Date"] >= "2023-01-01") & (dividends["Date"] < "2024-01-01")]

    # Stack together
    data = pd.concat([data, dividends])

In [20]:
data.groupby("name").sum(numeric_only=True).reset_index()

,name,Dividends
0,2222.SR,0.910764
1,AAPL,0.950000
2,JPM,4.050000
3,MSFT,2.790000


In [26]:
# Get trailing annual dividend yield
for stock in stocks:
    info = yf.Ticker(stock).info
    div = info.get('trailingAnnualDividendYield')
    if div:
        div = round(div * 100, 1)
    print(stock, div)

2222.SR 4.6
BRK-B None
AAPL 0.5
MSFT 0.7
GOOG None
JPM 2.2


In [27]:
print("Q5) Stock 2222.SR (Saudi Arabian Oil Company) has the largest dividend yield, namely 4.6%")

Q5) Stock 2222.SR (Saudi Arabian Oil Company) has the largest dividend yield, namely 4.6%
